#Dependencies#

In [ ]:
!pip install transformers
!pip install datasets transformers[SentencePiece]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

FOLDERNAME = 'CS685-Project/s2s-decipherment-multilingual'

import sys
sys.path.append('/content/drive/My Drive/{}/code'.format(FOLDERNAME))

%load_ext autoreload
%autoreload 2

%cd /content/drive/My\Drive/$FOLDERNAME

In [ ]:
from datasets import *
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, AdamW, DataCollatorWithPadding, get_scheduler
import pandas as pd
import torch
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from preprocess import *
from models import *
from data_utils import *
from transformers import logging
from datetime import date
import numpy as np
import torch.nn.functional.one_hot(

date = date.today()
model_lib = {}
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cpu')

#Code

In [ ]:
df_train = pd.read_csv('/content/drive/My Drive/CS685-Project/s2s-decipherment-multilingual/datasets/gigaword_eng_5_d1/data/afp_eng/english.dev', names = ['text'])
display(df_train)

,text
0,j a p a n _ a n d _ t h e _ u n i t e d _ s t ...
1,d u r i n g _ a _ w o r k i n g l e v e l _ m ...
2,e i s u k e _ s a k a k i b a r a _ d e p u t ...
3,o f f i c i a l s _ s a i d _ t h e y _ w e r ...
4,i n _ w a s h i n g t o n _ t h e _ t w o _ n ...
...,...
99060,i n _ a _ r a d i o _ i n t e r v i e w _ f r ...
99061,h e _ s a i d _ h i s _ p a s s p o r t _ h a ...
99062,h e _ h a d _ p l a n n e d _ t o _ t r a v e ...
99063,w a t t a n a _ s a i d _ h e _ h a d _ r e c ...


In [ ]:
N = 1000
Tx = 256
data_in = torch.randint(0, 26, size = (N, Tx))
data_in[torch.rand(data.size())<=0.2] = -2
data_in = data_in + 97

def enc_func(x):
  if x!=95:
    return x+6
  else:
    return x

def freq_enc(x):
  
  print("Now")
  print(x)
  min = np.amin(x)
  print("min = ", min)
  max = np.amax(x)
  print("max = ", max)

  freq = np.zeros(max-min+1)
  print ('freq')
  print(freq)
  x -= max
  
  for i in range(freq.shape[0]):
    freq[i] = np.sum(x==i)
  freq_rank = np.arange(max-min+1)[np.argsort(-freq)]
  
  for i in range(x.shape[0]):
    x[i] = freq_rank[x[i]]
  
  return x

encryption = np.vectorize(enc_func)
conv_to_chr = np.vectorize(lambda x: chr(x))
conv_to_freq = freq_enc

chr_data_in = conv_to_chr(data_in)
int_enc_in = encryption(data_in)
chr_enc_in = conv_to_chr(int_enc_in)

print(data_in)
print(chr_data_in)
print(int_enc_in)
print(chr_enc_in)

int_freq_in = conv_to_freq(int_enc_in)


labels = []
text = []
freq_strings = []


for i in range(N):

  seq = chr_data_in[i]
  # print(type(seq))
  label = ' '.join(seq).replace(' _ ', '_').replace(' _', '_').replace('_ ','_')
  # print(label)
  labels.append(label)

  seq = chr_enc_in[i]
  text_i = ' '.join(seq).replace(' _ ', '_').replace(' _', '_').replace('_ ','_')
  text.append(text_i)

  seq = int_freq_in[i]
  freq_string = np.array2string(seq, separator=' ')
  freq_strings.append(freq_string)

print (labels)
print (text)
print (freq_strings)

# get_data = np.vectorize(get_data_func)





tensor([[105,  95,  95,  ..., 100,  95, 103],
        [113, 107,  99,  ..., 120, 100,  95],
        [118, 116,  99,  ..., 113, 106, 119],
        ...,
        [118, 122, 114,  ..., 107,  95, 111],
        [118, 104,  97,  ...,  95, 122, 106],
        [120, 103, 108,  ..., 121, 122, 103]])
[['i' '_' '_' ... 'd' '_' 'g']
 ['q' 'k' 'c' ... 'x' 'd' '_']
 ['v' 't' 'c' ... 'q' 'j' 'w']
 ...
 ['v' 'z' 'r' ... 'k' '_' 'o']
 ['v' 'h' 'a' ... '_' 'z' 'j']
 ['x' 'g' 'l' ... 'y' 'z' 'g']]
[[111  95  95 ... 106  95 109]
 [119 113 105 ... 126 106  95]
 [124 122 105 ... 119 112 125]
 ...
 [124 128 120 ... 113  95 117]
 [124 110 103 ...  95 128 112]
 [126 109 114 ... 127 128 109]]
[['o' '_' '_' ... 'j' '_' 'm']
 ['w' 'q' 'i' ... '~' 'j' '_']
 ['|' 'z' 'i' ... 'w' 'p' '}']
 ...
 ['|' '\x80' 'x' ... 'q' '_' 'u']
 ['|' 'n' 'g' ... '_' '\x80' 'p']
 ['~' 'm' 'r' ... '\x7f' '\x80' 'm']]
Now
[[111  95  95 ... 106  95 109]
 [119 113 105 ... 126 106  95]
 [124 122 105 ... 119 112 125]
 ...
 [124 128 120 ... 11